<a href="https://colab.research.google.com/github/acredsfan/autonomous_mower/blob/main/Dual_Model_Vision_Training_7_7_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Cell 1: Final Combined Installation and Verification
# This is the ONLY cell you need for installations.

# 1. Uninstall to ensure a clean slate
print("Uninstalling all relevant libraries...")
!pip uninstall -y torch torchvision torchaudio ultralytics fiftyone tflite-runtime roboflow datasets

# 2. Install GPU-enabled PyTorch
print("\nInstalling GPU-enabled PyTorch...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# 3. Install all other dependencies in one go
print("\nInstalling Ultralytics and other libraries...")
!pip install ultralytics roboflow fiftyone datasets tflite-runtime --quiet

# 4. Verify the installation
print("\nVerifying the environment...")
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available? {torch.cuda.is_available()}")

print("\nRunning Ultralytics checks:")
import ultralytics
ultralytics.checks()

Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 44.6/235.7 GB disk)


In [3]:
# @title Cell 2: Class Setup and Directory Initialization
# ----------------------------
# CLASS SETUP & DIRECTORY INIT
# ----------------------------
import os
MASTER_CLASS_LIST = ["grass", "dirt", "sand", "mulch", "pavement", "concrete", "gravel", "tree", "shrub", "flower", "planter", "stump", "rock", "hill", "water_feature", "ditch", "pool", "lake", "river", "fountain", "waterfall", "field", "curb", "edging", "fence", "gate", "retaining_wall", "railing", "bench", "bridge", "stairs", "path", "sign", "pole", "lamp_post", "streetlight", "traffic_light", "person", "animal", "dog", "cat", "bicycle", "toy", "tool", "hose", "sprinkler", "swing_set", "slide", "sandbox", "trampoline", "furniture", "decoration", "vehicle", "car", "bus", "truck", "mailbox", "trash_bin", "recycling_bin"]
master_index = {name: idx for idx, name in enumerate(MASTER_CLASS_LIST)}
BASE_DIR = "/content/mower_dataset"
for split in ["train", "val"]:
    os.makedirs(f"{BASE_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{BASE_DIR}/labels/{split}", exist_ok=True)


In [4]:
# @title Cell 3: Kaggle setup for COCO
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to kaggle.json in your Google Drive
kaggle_drive_path = "/content/drive/MyDrive/kaggle.json"
kaggle_colab_path = "/root/.kaggle/kaggle.json"

# Create the .kaggle directory if it doesn't exist
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")

# Check if the file exists in Google Drive before copying
if os.path.exists(kaggle_drive_path):
    shutil.copy(kaggle_drive_path, kaggle_colab_path)
    os.chmod(kaggle_colab_path, 0o600)
    print("kaggle.json copied from Google Drive.")
else:
    print(f"Error: {kaggle_drive_path} not found in your Google Drive.")
    print("Please upload kaggle.json to the root of your MyDrive or manually upload it.")
    # Fallback to manual upload if the file is not in Drive
    from google.colab import files
    uploaded = files.upload()  # upload kaggle.json
    for fn in uploaded.keys():
        shutil.move(fn, kaggle_colab_path)
    os.chmod(kaggle_colab_path, 0o600)
    print("kaggle.json uploaded manually.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
kaggle.json copied from Google Drive.


In [ ]:
# @title Cell 4: Download COCO 2017 (YOLOv8 format from Kaggle)
!kaggle datasets download -d paragmraw/coco-2017-dataset-yolov8-format -p /content
!unzip -q /content/coco-2017-dataset-yolov8-format.zip -d /content/coco2017
print("COCO 2017 dataset downloaded and extracted!")

Dataset URL: https://www.kaggle.com/datasets/paragmraw/coco-2017-dataset-yolov8-format
License(s): CC0-1.0
 99% 5.81G/5.86G [00:13<00:00, 798MB/s]
100% 5.86G/5.86G [00:13<00:00, 456MB/s]


In [ ]:
# @title Cell 5: COCO mapping and filtering
COCO_TO_MASTER = {
    0: 36,    # person
    1: 40,    # bicycle
    2: 51,    # car
    3: 50,    # motorcycle -> vehicle
    5: 52,    # bus
    7: 53,    # truck
    15: 39,   # cat
    16: 38,   # dog
    57: 48,   # chair -> furniture
    60: 48,   # dining table -> furniture
    58: 9,    # potted plant -> planter
    77: 42,   # teddy bear -> toy
}
COCO_IMG_LIMIT = None  # Adjust for RAM/time; set to None for full dataset

for split in ["train", "val"]:
    img_dir = f"/content/coco2017/{split}/images"
    lbl_dir = f"/content/coco2017/{split}/labels"
    imgs = glob.glob(f"{img_dir}/*.jpg")
    if COCO_IMG_LIMIT:
        imgs = random.sample(imgs, min(COCO_IMG_LIMIT, len(imgs)))
    for img_path in tqdm(imgs, desc=f"COCO {split}"):
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(lbl_dir, base + ".txt")
        if not os.path.exists(lbl_path): continue
        new_lines = []
        with open(lbl_path, "r") as f:
            for line in f:
                arr = line.strip().split()
                if not arr: continue
                cls = int(arr[0])
                if cls in COCO_TO_MASTER:
                    new_cls = COCO_TO_MASTER[cls]
                    # Ensure we only use the 4 bounding box values, ignoring segmentation data
                    new_lines.append(" ".join([str(new_cls)] + arr[1:5]))
        if new_lines:
            out_img = f"{BASE_DIR}/images/{split}/{base}.jpg"
            out_lbl = f"{BASE_DIR}/labels/{split}/{base}.txt"
            shutil.copy(img_path, out_img)
            with open(out_lbl, "w") as f:
                f.write("\n".join(new_lines))

print("COCO dataset processed!")

In [ ]:
# @title Cell 6: OpenImages dataset download

import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.openimages as fouo

# 1. Full valid OI class set
oi_valid_classes = set(fouo.get_classes())

# 2. Define desired yard/obstacle classes
OI_YARD_CLASSES = [
    "Flower", "Fountain", "Stairs", "Person", "Dog", "Cat", "Bicycle",
    "Car", "Bus", "Truck", "Motorcycle", "Bench", "Tree", "Lamp", "Wheelchair", "Table", "Chair",
]

# 3. Only request OI classes that exist
oi_classes_to_load = [c for c in OI_YARD_CLASSES if c in oi_valid_classes]
print("Attempting to load from OpenImages:", oi_classes_to_load)

# 4. Download data
oi_dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",
    label_types=["detections"],
    classes=oi_classes_to_load,
    max_samples=2500,  # Increased sample size for more variety
    shuffle=True,
)

# 5. Export to YOLO format for processing
oi_dataset.export(
    export_dir="/content/fo_openimages_yolo",
    dataset_type=fo.types.YOLOv5Dataset
)
print("OpenImages export complete.")

In [ ]:
# @title Cell 7: Process and Integrate OpenImages Data
import yaml

print("Processing and integrating OpenImages dataset...")

# Directory where fiftyone exported the data
oi_export_dir = "/content/fo_openimages_yolo"
oi_img_dir = f"{oi_export_dir}/data/images"
oi_lbl_dir = f"{oi_export_dir}/data/labels"

# Check if the export directory exists
if not os.path.isdir(oi_export_dir):
    print("OpenImages export directory not found. Skipping integration.")
else:
    # Read the class mapping from the exported dataset.yaml
    with open(f"{oi_export_dir}/dataset.yaml", 'r') as f:
        oi_yaml = yaml.safe_load(f)
    oi_names = oi_yaml['names']

    # Create a mapping from OI class name to our master class index
    OI_NAME_TO_MASTER = {
        "Tree": master_index["tree"],
        "Flower": master_index["flower"],
        "Person": master_index["person"],
        "Car": master_index["car"],
        "Bus": master_index["bus"],
        "Truck": master_index["truck"],
        "Bicycle": master_index["bicycle"],
        "Cat": master_index["cat"],
        "Dog": master_index["dog"],
        "Stairs": master_index["stairs"],
        "Fountain": master_index["fountain"],
        "Bench": master_index["bench"],
        "Lamp": master_index["lamp_post"],
        "Chair": master_index["furniture"],
        "Table": master_index["furniture"],
        "Motorcycle": master_index["vehicle"],
        "Wheelchair": master_index["vehicle"]
    }

    # Map from the numeric index in the OI export to our master index
    oi_idx_to_master_idx = {
        oi_idx: OI_NAME_TO_MASTER.get(name)
        for oi_idx, name in enumerate(oi_names)
        if OI_NAME_TO_MASTER.get(name) is not None
    }

    # Process and copy the files
    imgs = glob.glob(f"{oi_img_dir}/*.jpg")
    for img_path in tqdm(imgs, desc="OpenImages Process"):
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(oi_lbl_dir, base + ".txt")

        if not os.path.exists(lbl_path):
            continue

        new_lines = []
        with open(lbl_path, 'r') as f:
            for line in f:
                arr = line.strip().split()
                if not arr: continue
                oi_cls_idx = int(arr[0])
                if oi_cls_idx in oi_idx_to_master_idx:
                    master_cls_idx = oi_idx_to_master_idx[oi_cls_idx]
                    # Ensure we only take the 4 bounding box coordinates
                    new_lines.append(f"{master_cls_idx} " + " ".join(arr[1:5]))

        if new_lines:
            # Copy to train split
            out_img = f"{BASE_DIR}/images/train/oi_{base}.jpg"
            out_lbl = f"{BASE_DIR}/labels/train/oi_{base}.txt"
            shutil.copy(img_path, out_img)
            with open(out_lbl, 'w') as f:
                f.write("\n".join(new_lines))

    print("OpenImages dataset processed and added to training set!")

In [ ]:
# @title Cell 8: Roboflow Fence dataset (requires API key)
from roboflow import Roboflow
from google.colab import userdata
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
fence_ds = rf.workspace("uji-thesis").project("broken-fence-detection").version(1).download("yolov8", location=f"{BASE_DIR}/fence")
for split in ["train", "valid"]:
    img_dir = f"{BASE_DIR}/fence/{split}/images"
    lbl_dir = f"{BASE_DIR}/fence/{split}/labels"
    imgs = glob.glob(f"{img_dir}/*.jpg")
    for img_path in tqdm(imgs, desc=f"Fence {split}"):
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(lbl_dir, base + ".txt")
        if not os.path.exists(lbl_path): continue
        new_lines = []
        with open(lbl_path, "r") as f:
            for line in f:
                arr = line.strip().split()
                if not arr: continue
                # Ensure we only use the 4 bounding box values, ignoring segmentation data
                new_lines.append(f"{master_index['fence']} " + " ".join(arr[1:5]))
        if new_lines:
          # Add all Roboflow data to the training set
          out_img = f"{BASE_DIR}/images/train/{base}_rf.jpg"
          out_lbl = f"{BASE_DIR}/labels/train/{base}_rf.txt"
          shutil.copy(img_path, out_img)
          with open(out_lbl, "w") as f:
              f.write("\n".join(new_lines))

print("Fence dataset processed!")

In [ ]:
# @title Cell 9: ADE20K via Kaggle (awsaf49/ade20k-dataset)
if not os.path.exists("/content/ade20k-dataset.zip"):
    !kaggle datasets download -d awsaf49/ade20k-dataset -p /content
    !unzip -q /content/ade20k-dataset.zip -d /content/ade20k

import os
import glob
import shutil
from PIL import Image
import numpy as np
from tqdm import tqdm

# ADE20K category mapping for yard/obstacle/terrain, index: master_class
ADE_TO_MASTER = {
    1: 26,      # wall -> retaining_wall
    5: 7,       # tree -> tree
    7: 4,       # road, route -> pavement
    10: 0,      # grass -> grass
    12: 4,      # sidewalk, pavement -> pavement
    13: 36,     # person -> person
    14: 1,      # earth, ground -> dirt
    16: 48,     # table -> furniture
    18: 8,      # plant -> shrub
    21: 51,     # car -> car
    22: 14,     # water -> water_feature
    30: 21,     # field -> field
    33: 24,     # fence -> fence
    35: 12,     # rock, stone -> rock
    39: 27,     # railing, rail -> railing
    44: 31,     # signboard, sign -> sign
    47: 2,      # sand -> sand
    53: 30,     # path -> path
    54: 29,     # stairs, steps -> stairs
    60: 29,     # stairway, staircase -> stairs
    61: 18,     # river -> river
    62: 28,     # bridge -> bridge
    67: 9,      # flower -> flower
    69: 13,     # hill -> hill
    70: 28,     # bench -> bench
    73: 7,      # palm tree -> tree
    80: 8,      # shrub -> shrub
    81: 52,     # bus -> bus
    83: 33,     # light -> lamp_post
    84: 53,     # truck -> truck
    88: 34,     # streetlight -> streetlight
    94: 32,     # pole -> pole
    95: 1,      # land, ground, soil -> dirt
    105: 19,    # fountain -> fountain
    109: 42,    # plaything, toy -> toy
    110: 16,    # swimming pool -> pool
    114: 20,    # waterfall, falls -> waterfall
    127: 37,    # animal -> animal
    128: 40,    # bicycle -> bicycle
    129: 17,    # lake -> lake
    137: 35,    # traffic light -> traffic_light
    139: 55,    # ashcan, trash can -> trash_bin
}
print("ADE20K category mapping loaded!")

img_dir = "/content/ade20k/ADEChallengeData2016/images/training"
mask_dir = "/content/ade20k/ADEChallengeData2016/annotations/training"

imgs = sorted(glob.glob(f"{img_dir}/*.jpg"))
masks = sorted(glob.glob(f"{mask_dir}/*.png"))

print(f"Found {len(imgs)} images and {len(masks)} masks!")

assert len(imgs) == len(masks), "Mismatch in image and mask counts!"

print("Processing ADE20K images...")

for img_path, mask_path in tqdm(zip(imgs, masks), total=len(imgs), desc="ADE20K images"):
    img = Image.open(img_path).convert("RGB")
    mask = np.array(Image.open(mask_path))
    W, H = img.size # Correctly get width and height from PIL Image
    objs = []
    for ade_class, master_id in ADE_TO_MASTER.items():
        ys, xs = np.where(mask == ade_class)
        if len(xs) < 2 or len(ys) < 2: # Need at least 2 points to form a box
            continue
        xmin, xmax = xs.min(), xs.max()
        ymin, ymax = ys.min(), ys.max()
        # Skip zero-area boxes
        if xmin >= xmax or ymin >= ymax:
            continue
        x_c = (xmin + xmax) / 2.0 / W
        y_c = (ymin + ymax) / 2.0 / H
        bw = (xmax - xmin) / W
        bh = (ymax - ymin) / H
        objs.append(f"{master_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")
    if objs:
        base = os.path.splitext(os.path.basename(img_path))[0]
        out_img = f"{BASE_DIR}/images/train/ade_{base}.jpg"
        out_lbl = f"{BASE_DIR}/labels/train/ade_{base}.txt"
        img.save(out_img)
        with open(out_lbl, "w") as f:
            f.write("\n".join(objs))

print("ADE20K Kaggle dataset processed!")

In [ ]:
# ----------------------------
# @title Cell 10: WRITE DATA CONFIG FILE
# ----------------------------
with open(f"{BASE_DIR}/data.yaml", "w") as f:
    f.write(f"path: {BASE_DIR}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write(f"nc: {len(MASTER_CLASS_LIST)}\n")
    f.write("names: " + str(MASTER_CLASS_LIST) + "\n")


In [ ]:
# ----------------------------
# @title Cell 11: TRAIN MODELS WITH CHECKPOINT RESUME
# ----------------------------
import torch
import os
import shutil
import random
import glob
import gc
import re
import time # Import time for modification time
from tqdm import tqdm # Import tqdm here as it's used later
from ultralytics import YOLO
from google.colab import drive

# Mount Google Drive for automated backups
drive.mount('/content/drive')
BACKUP_DIR = "/content/drive/MyDrive/mower_model_checkpoints"
os.makedirs(BACKUP_DIR, exist_ok=True)

# --- (IMPROVED) Define the backup callback function ---
def backup_checkpoint_callback(trainer):
    """
    A callback to save model-specific checkpoints to Google Drive every 5 epochs.
    """
    epoch = trainer.epoch
    # Get the actual run name (which might have suffixes like '_2')
    run_name = os.path.basename(trainer.save_dir)
    model_name = trainer.args.name # Gets the base model name (e.g., 'pi_model_yolov8n')

    # Use the run_name from save_dir, which reflects the actual directory name
    backup_file_name = f"{run_name}_epoch_{epoch+1}.pt"
    src_path = trainer.last
    dst_path = os.path.join(BACKUP_DIR, backup_file_name)

    if (epoch + 1) % 5 == 0:
        if os.path.exists(src_path):
            try:
                shutil.copy2(src_path, dst_path)
                print(f"✅ [Backup] Epoch {epoch+1} for '{run_name}' saved to Google Drive at {dst_path}")
            except Exception as e:
                print(f"❌ [Backup] Failed to save checkpoint {backup_file_name} to Google Drive: {e}")


# --- (IMPROVED) Function to find the latest checkpoint by modification date ---
def find_latest_checkpoint(backup_dir, base_model_name):
    """
    Finds the latest checkpoint file for a given base model name in the backup directory
    based on the file modification date, handling potential suffixes added by the
    training framework. Returns the file path or None if no checkpoint is found.
    """
    latest_checkpoint = None
    latest_mtime = 0 # Use modification time
    # Match files starting with the base name, potentially followed by _\d+ and _epoch_\d+
    # The epoch number is not used for sorting here, only for filtering by base name
    pattern = re.compile(rf"^{re.escape(base_model_name)}(_\d+)?_epoch_\d+\.pt$")

    if not os.path.exists(backup_dir):
        print(f"Backup directory not found: {backup_dir}")
        return None

    print(f"Searching for latest checkpoint matching pattern '{pattern.pattern}' in {backup_dir} by modification date.")

    matching_files = []
    for filename in os.listdir(backup_dir):
        if pattern.match(filename):
            file_path = os.path.join(backup_dir, filename)
            matching_files.append((file_path, os.path.getmtime(file_path)))

    if matching_files:
        # Find the file with the latest modification time
        latest_checkpoint, latest_mtime = max(matching_files, key=lambda item: item[1])
        # Optionally, parse the epoch number from the latest file found for reporting
        epoch_match = re.search(r"_epoch_(\d+)\.pt$", os.path.basename(latest_checkpoint))
        latest_epoch_str = epoch_match.group(1) if epoch_match else "N/A"
        print(f"🔎 Found latest checkpoint for '{base_model_name}' (modified {time.ctime(latest_mtime)}): {latest_checkpoint} (Epoch: {latest_epoch_str})")
    else:
        print(f"👍 No existing checkpoint found matching pattern '{pattern.pattern}'. Starting new training.")

    return latest_checkpoint

# --- Clear GPU function ---
def clear_gpu():
    torch.cuda.empty_cache()
    gc.collect()
    print("Cleared GPU cache.")

# --- Auto-create validation set if empty ---
# (This part of the code remains unchanged)
train_img_dir = f"{BASE_DIR}/images/train"
val_img_dir = f"{BASE_DIR}/images/val"
train_lbl_dir = f"{BASE_DIR}/labels/train"
val_lbl_dir = f"{BASE_DIR}/labels/val"

# Ensure BASE_DIR is defined (from Cell 2)
if 'BASE_DIR' not in globals():
    print("Error: BASE_DIR is not defined. Please run Cell 2 first.")
else:
    if not os.path.exists(val_img_dir) or not os.listdir(val_img_dir):
        print("Validation set is empty or not found. Creating one...")
        all_imgs = [img for img in os.listdir(train_img_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
        # Calculate number of validation images, ensure at least 1 and at most 1500
        num_val = min(1500, max(1, len(all_imgs) // 10))
        if len(all_imgs) > num_val:
            val_imgs_to_move = random.sample(all_imgs, num_val)

            for img_name in tqdm(val_imgs_to_move, desc="Moving validation images"):
                src_img = os.path.join(train_img_dir, img_name)
                dst_img = os.path.join(val_img_dir, img_name)
                shutil.move(src_img, dst_img)

                label_name = os.path.splitext(img_name)[0] + ".txt"
                src_lbl = os.path.join(train_lbl_dir, label_name)
                dst_lbl = os.path.join(val_lbl_dir, label_name)
                if os.path.exists(src_lbl):
                    shutil.move(src_lbl, dst_lbl)
                elif not os.path.exists(src_lbl):
                    # Create an empty label file if it doesn't exist in the source
                    # This prevents errors later if the image had no annotations
                    with open(dst_lbl, 'w') as f:
                        pass # Create an empty file
            print(f"Moved {num_val} images/labels to validation set.")
        else:
            print(f"Not enough images ({len(all_imgs)}) to create a validation set of size {num_val}. Skipping validation set creation.")
    else:
        print("Validation set already exists and is not empty.")


# --- Train Pi model with resume logic ---
pi_model_name = "pi_model_yolov8n"
# Look for the latest checkpoint by modification date
pi_checkpoint_path = find_latest_checkpoint(BACKUP_DIR, pi_model_name)

# Load from checkpoint if it exists, otherwise start new
total_epochs_pi = 50 # Set your desired total epochs here
if pi_checkpoint_path:
    print(f"Attempting to resume Pi model training from {pi_checkpoint_path}.")
else:
    print("Starting new Pi model training.")

model_pi = YOLO(pi_checkpoint_path) if pi_checkpoint_path else YOLO("yolov8n.yaml")
model_pi.add_callback("on_train_epoch_end", backup_checkpoint_callback)

# Adjust epochs and resume flag based on whether a checkpoint was found
pi_train_args = {
    "data": f"{BASE_DIR}/data.yaml",
    "epochs": total_epochs_pi, # Set total epochs
    "imgsz": 640,
    "batch": -1,
    "workers": 6,
    "patience": 10,
    "seed": 42,
    "project": "mower_model",
    "name": pi_model_name,
    "resume": bool(pi_checkpoint_path) # Explicitly tell trainer to resume
}

# Only set device if CUDA is available
if torch.cuda.is_available():
    pi_train_args["device"] = 0 # Use GPU 0

model_pi.train(**pi_train_args)


clear_gpu()
del model_pi
gc.collect()

# --- Train Coral model with resume logic ---
coral_model_name = "coral_model_yolov8n"
# Look for the latest checkpoint by modification date
coral_checkpoint_path = find_latest_checkpoint(BACKUP_DIR, coral_model_name)

# Load from checkpoint if it exists, otherwise start new
total_epochs_coral = 50 # Set your desired total epochs here
if coral_checkpoint_path:
     print(f"Attempting to resume Coral model training from {coral_checkpoint_path}.")
else:
    print("Starting new Coral model training.")

model_coral = YOLO(coral_checkpoint_path) if coral_checkpoint_path else YOLO("yolov8n.yaml")
model_coral.add_callback("on_train_epoch_end", backup_checkpoint_callback)

# Adjust epochs and resume flag based on whether a checkpoint was found
coral_train_args = {
    "data": f"{BASE_DIR}/data.yaml",
    "epochs": total_epochs_coral, # Set total epochs
    "imgsz": 640,
    "batch": -1,
    "workers": 6,
    "patience": 10,
    "seed": 42,
    "project": "mower_model",
    "name": coral_model_name,
    "resume": bool(coral_checkpoint_path) # Explicitly tell trainer to resume
}

# Only set device if CUDA is available
if torch.cuda.is_available():
    coral_train_args["device"] = 0 # Use GPU 0

model_coral.train(**coral_train_args)

In [ ]:
# ----------------------------
# @title Cell 12: CREATE REPRESENTATIVE SET
# ----------------------------
rep_data_dir = "/content/valid_images/"
os.makedirs(rep_data_dir, exist_ok=True)
val_imgs = glob.glob(os.path.join(f"{BASE_DIR}/images/val", "*.jpg"))
for img_path in random.sample(val_imgs, min(150, len(val_imgs))):
    shutil.copy(img_path, rep_data_dir)


In [ ]:
# ----------------------------
# @title Cell 13: EXPORT MODELS (with GPU Check)
# ----------------------------
import tensorflow as tf
from ultralytics import YOLO
import os
import yaml

# --- (NEW) GPU VERIFICATION STEP ---
print("Verifying GPU availability...")
!nvidia-smi
print(f"TensorFlow version: {tf.__version__}")
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"✅ TensorFlow has detected the following GPUs: {gpu_devices}")
else:
    print("❌ WARNING: TensorFlow did NOT detect a GPU. The export will be very slow.")
# ---

# Save label file
with open("labels.txt", "w") as f:
    for name in MASTER_CLASS_LIST:
        f.write(name + "\n")

# Define the base path where training results are saved
train_output_base = "mower_model"

# --- Export TFLite for Pi model ---
pi_model_path = os.path.join(train_output_base, "pi_model_yolov8n", "weights", "best.pt")
model_pi = YOLO(pi_model_path)
model_pi.export(format="tflite", imgsz=640, int8=False, name="pi_best_float32")

# --- Export TFLite and ONNX for Coral model ---
coral_model_path = os.path.join(train_output_base, "coral_model_yolov8n", "weights", "best.pt")
model_coral = YOLO(coral_model_path)

# --- Create a temporary YAML file for INT8 quantization ---
rep_yaml_path = "/content/rep_data.yaml"
with open(f"{BASE_DIR}/data.yaml", 'r') as f:
    data_config = yaml.safe_load(f)
data_config['val'] = rep_data_dir
with open(rep_yaml_path, 'w') as f:
    yaml.dump(data_config, f)
# ---

# Correctly export the quantized TFLite model for Coral
print(f"Starting INT8 TFLite export using configuration from '{rep_yaml_path}'...")
model_coral.export(
    format="tflite",
    imgsz=640,
    int8=True,
    data=rep_yaml_path,
    name="coral_best_int8"
)

# Export the ONNX model
model_coral.export(format="onnx", imgsz=640, name="coral_best")

In [ ]:
# ----------------------------
# @title Cell 14: COMPILE FOR CORAL
# ----------------------------
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update && sudo apt-get install edgetpu-compiler -y
!edgetpu_compiler /content/mower_model/coral_model_yolov8n/weights/coral_best_int8.tflite --out_dir=/content/autonomous_mower/models/

In [ ]:
# ----------------------------
# @title Cell 15: PUSH TO GITHUB
# ----------------------------
from google.colab import userdata
import os
import shutil

GITHUB_REPO_URL = "https://github.com/acredsfan/autonomous_mower.git"
GITHUB_BRANCH = "code_rebuild"
# Get the GitHub Personal Access Token from Colab secrets
GITHUB_PAT = userdata.get('GITHUB_PAT')

if not GITHUB_PAT:
    print("Error: GitHub Personal Access Token not found in secrets. Please add it as 'GITHUB_PAT'.")
else:
    !git config --global user.email "you@example.com"
    !git config --global user.name "Autonomous Mower Trainer"

    # Clone the repository using the PAT
    !git clone -b $GITHUB_BRANCH https://$GITHUB_PAT@github.com/acredsfan/autonomous_mower.git push_dir

    # Check if the clone was successful before proceeding
    if os.path.exists("push_dir"):
        # Copy the generated files to the cloned repository
        !cp -r pi_best_float32.tflite coral_best_int8.tflite coral_best_edgetpu.tflite coral_best.onnx labels.txt push_dir/models/

        # Change directory to the cloned repository
        %cd push_dir

        # Add, commit, and push the changes
        !git add models/*
        !git commit -m "Add full dataset trained YOLOv8 models for Pi and Coral, compiled for EdgeTPU"
        !git push origin $GITHUB_BRANCH

        # Change back to the original directory
        %cd ..
    else:
        print("Error: Repository cloning failed. Please check your PAT and repository URL.")

In [ ]:
# prompt: I want the final models saved to my Google Drive folder

# @title Cell 16: SAVE MODELS TO GOOGLE DRIVE
# ----------------------------

# Define the destination folder in your Google Drive
DRIVE_SAVE_DIR = "/content/drive/MyDrive/Mower_Trained_Models"
os.makedirs(DRIVE_SAVE_DIR, exist_ok=True)

# Define the files to save from the current Colab environment
files_to_save = [
    "pi_best_float32.tflite",
    "coral_best_int8.tflite",
    "coral_best_edgetpu.tflite",
    "coral_best.onnx",
    "labels.txt"
]

print(f"Saving models to Google Drive folder: {DRIVE_SAVE_DIR}")

for file_name in files_to_save:
    src_path = file_name
    dst_path = os.path.join(DRIVE_SAVE_DIR, file_name)
    if os.path.exists(src_path):
        shutil.copy2(src_path, dst_path)
        print(f"✅ Saved {file_name} to Google Drive")
    else:
        print(f"❌ Error: {file_name} not found. Skipping save to Google Drive.")

print("Model saving to Google Drive complete.")
